# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
# ... ENTER YOUR CODE HERE ...

import numpy as np

def jacobi_iteration(matrix_A, vector_b, num_iterations):
    n = matrix_A.shape[0]
    solution_vector = np.zeros(n)

    norms_diff = []
    norms_B = []

    for _ in range(num_iterations):
        new_solution_vector = np.zeros(n)
        for i in range(n):
            new_solution_vector[i] = (vector_b[i] - np.dot(matrix_A[i, :i], solution_vector[:i]) - np.dot(matrix_A[i, i+1:], solution_vector[i+1:])) / matrix_A[i, i]
        norms_diff.append(np.linalg.norm(new_solution_vector - solution_vector))
        norms_B.append(np.linalg.norm(matrix_A - np.diag(np.diagonal(matrix_A))))

        solution_vector = new_solution_vector

    return solution_vector, norms_diff, norms_B

# Example
original_matrix_A = np.array([[4, 1, 1],
                              [1, 5, 2],
                              [1, 2, 6]])

right_hand_side_b = np.array([1, 2, 3])

num_iterations = 10

# Original matrix A
solution_orig, diff_orig, norm_B_orig = jacobi_iteration(original_matrix_A, right_hand_side_b, num_iterations)

# Modified matrix A with smaller diagonal elements
modified_matrix_A = np.array([[0.4, 0.1, 0.1],
                               [0.1, 0.5, 0.2],
                               [0.1, 0.2, 0.6]])

solution_modified, diff_modified, norm_B_modified = jacobi_iteration(modified_matrix_A, right_hand_side_b, num_iterations)

print("Original A:")
print("Solution vector:", solution_orig)
print("Norms of differences between consecutive solutions:", diff_orig)
print("Norms of matrix B:", norm_B_orig)
print()

print("Modified A with smaller diagonal elements:")
print("Solution vector:", solution_modified)
print("Norms of differences between consecutive solutions:", diff_modified)
print("Norms of matrix B:", norm_B_modified)


Original A:
Solution vector: [0.09229869 0.21595253 0.41187576]
Norms of differences between consecutive solutions: [0.687386354243376, 0.3791437722025775, 0.19777451035066282, 0.10704482121989833, 0.05698190978314714, 0.03064512627199415, 0.01638126971629196, 0.00879260153435083, 0.004706802221473611, 0.002524255646749693]
Norms of matrix B: [3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544]

Modified A with smaller diagonal elements:
Solution vector: [0.92298693 2.15952527 4.11875764]
Norms of differences between consecutive solutions: [6.87386354243376, 3.7914377220257753, 1.9777451035066285, 1.0704482121989833, 0.5698190978314713, 0.3064512627199413, 0.16381269716291957, 0.08792601534350843, 0.04706802221473645, 0.025242556467497023]
Norms of matrix B: [0.3464101615137755, 0.3464101615137755, 0.3464101615137755, 0.3464101615137755, 0.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [9]:
# ... ENTER YOUR CODE HERE ...

import numpy as np

def gauss_seidel_iteration(coeff_matrix_A, rhs_vector_b, num_iterations):
    size = coeff_matrix_A.shape[0]
    solution_vector = np.zeros(size)  # Initial guess for the solution vector

    norms_diff = []
    norms_B = []

    for _ in range(num_iterations):
        new_solution_vector = np.zeros(size)
        for i in range(size):
            new_solution_vector[i] = (rhs_vector_b[i] - np.dot(coeff_matrix_A[i, :i], new_solution_vector[:i]) - np.dot(coeff_matrix_A[i, i+1:], solution_vector[i+1:])) / coeff_matrix_A[i, i]
        norms_diff.append(np.linalg.norm(new_solution_vector - solution_vector))
        norms_B.append(np.linalg.norm(coeff_matrix_A - np.tril(coeff_matrix_A) @ np.triu(coeff_matrix_A, 1)))

        solution_vector = new_solution_vector

    return solution_vector, norms_diff, norms_B

# Example
matrix_size = 5  # Size of the matrix
random_matrix_A = np.random.rand(matrix_size, matrix_size)  # Random matrix
random_rhs_vector_b = np.random.rand(matrix_size)  # Random right-hand side vector
num_iterations = 10

solution, diff, norm_B = gauss_seidel_iteration(random_matrix_A, random_rhs_vector_b, num_iterations)

print("Solution vector:", solution)
print("Norms of differences between consecutive solutions:", diff)
print("Norms of iteration matrix B:", norm_B)


Solution vector: [ 4.28501194e+07 -3.62612565e+07  2.54593307e+07  1.68112976e+08
 -8.11728907e+08]
Norms of differences between consecutive solutions: [4.661445959830939, 24.753027118407942, 196.69065042788662, 1687.4358499325722, 14660.672533860792, 127672.58104321104, 1112339.1686998634, 9692030.4810567, 84450003.36658797, 735844415.4131263]
Norms of iteration matrix B: [2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035, 2.2158550725297035]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [10]:
# ... ENTER YOUR CODE HERE ...

import numpy as np

def minres_scheme(coeff_matrix_A, rhs_vector_b, num_iterations):
    size = coeff_matrix_A.shape[0]
    solution_vector = np.zeros(size)  # Initial guess for the solution vector
    residual_vector = rhs_vector_b - coeff_matrix_A @ solution_vector  # Initial residual vector
    search_direction_vector = residual_vector.copy()  # Initial search direction vector

    norms_residual = []
    norms_deviation = []
    tau_values = []

    for k in range(num_iterations):
        Ap = coeff_matrix_A @ search_direction_vector
        denom = np.dot(search_direction_vector, Ap)
        if denom == 0:
            break
        tau = np.dot(residual_vector, Ap) / denom
        solution_vector_new = solution_vector + tau * search_direction_vector
        residual_vector_new = residual_vector - tau * Ap

        norms_residual.append(np.linalg.norm(residual_vector_new))
        norms_deviation.append(np.linalg.norm(solution_vector_new - solution_vector))
        tau_values.append(tau)

        solution_vector = solution_vector_new
        residual_vector = residual_vector_new
        beta = np.linalg.norm(residual_vector)
        search_direction_vector = residual_vector + (beta / np.linalg.norm(residual_vector)) * search_direction_vector

    return solution_vector, norms_residual, norms_deviation, tau_values

# Example
matrix_size = 5  # Size of the matrix
random_matrix_A = np.random.rand(matrix_size, matrix_size)  # Random matrix
random_rhs_vector_b = np.random.rand(matrix_size)  # Random right-hand side vector
num_iterations = 10

# Obtain ground truth solution using a direct method
solution_ground_truth = np.linalg.solve(random_matrix_A, random_rhs_vector_b)

solution_minres, norms_residual, norms_deviation, tau_values = minres_scheme(random_matrix_A, random_rhs_vector_b, num_iterations)

print("Ground truth solution:", solution_ground_truth)
print("MINRES solution:", solution_minres)
print("Norms of residual:", norms_residual)
print("Norms of deviation from ground truth solution:", norms_deviation)
print("Iteration parameter tau values:", tau_values)


Ground truth solution: [ 0.29906416 -1.91622822  3.73195474 -3.07972545  1.71076669]
MINRES solution: [-954.1129047  -735.73752782  541.59138094 -955.21444971  291.24836457]
Norms of residual: [3.046101565484322, 3.859620802532755, 20.28532411938484, 35.23603916198933, 94.61365096081141, 179.2456106720324, 336.36450985322864, 691.9392243335232, 1130.9063365567815, 2534.3845752755274]
Norms of deviation from ground truth solution: [1.3531473766374686, 4.011688889592961, 12.350613098299679, 22.343099355055067, 70.88109685142987, 113.29918730286705, 255.08394235635404, 414.1681437873938, 789.5747714340074, 1288.5836618246133]
Iteration parameter tau values: [1.0, 1.682433366519159, 3.420967836498932, 1.1011595540824923, 2.5057639804298315, 1.236821658060584, 1.7953006339457596, 1.4428526462944995, 1.490054827063662, 1.6733036637409444]
